In [ ]:
#| hide
from GptQA.crowler import *
from GptQA.KnowledgeEmbedding import *

# GptQA

> A Q/A framework using Open.AI tools - This is an educational Repo

## Install

```sh
pip install GptQA
```

## How to use

### Running The Crawler

In [ ]:
crawl("https://yuval6957.github.io/reinautils/","/home/hd/GptQA")

https://yuval6957.github.io/reinautils/
https://yuval6957.github.io/./torchutils.html
HTTP Error 404: Not Found
https://yuval6957.github.io/./index.html
HTTP Error 404: Not Found
https://yuval6957.github.io/./parameters.html
HTTP Error 404: Not Found


### Creating a file with all the text files in the directory

In [ ]:
#| eval : false
# Set the text column to be the raw text with the newlines removed
import pickle
texts=text2data("/home/hd/GptQA/text",'txt',recursive=True)

print (texts[:5]) 

with open("/home/hd/GptQA/text_accum.pkl","wb") as f:
    pickle.dump(texts,f)

### Tokenizing and Embedding all the text

In [ ]:
import os
import glob
import pandas as pd
from tqdm.auto import tqdm
from typing import List, Dict, Set, Union, Callable
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import torch.nn.functional as F
from functools import partial
import transformers


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
tokenized = tokenize_data(texts, tokenizer, max_tokens = 500)
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2').to('cuda')

embedded = embed_data(tokenized, partial(run_embeddings,model=model))

with open("/home/hd/GptQA/embedding_all-mpnet-base-v2.pkl","wb") as f:
    pickle.dump(embedded,f)

### Asking a quation and getting the most relevant context

In [ ]:
question = 'What are the  language model I can use?'
# question = 'How do I get access to GPT4'
answers = top_scores(question, embedded,model,tokenizer)
print(answers)

### Asking a question ang getting an answer (using relevant models from Huggingface)

#### Load the models

In [ ]:
#| eval : false
# Loading the models for context creation
context_tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
context_model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2').to('cuda')

# Loading the models for QA
tokenizer = AutoTokenizer.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained("togethercomputer/RedPajama-INCITE-Chat-7B-v0.1", torch_dtype=torch.float16).to('cuda')

#### Asking a question and getting the answer

In [ ]:
#| eval : false
question="What is our newest embeddings model?"
answer = answer_question(question, embedded, context_model=context_model, context_tokenizer = context_tokenizer,
                         model = model, tokenizer = tokenizer, max_len = 1800, 
                    max_added_tokens = 150, 
                    temperature = 0.7,
                    debug = False) 
print (answer)